<a href="https://colab.research.google.com/github/ymuto0302/RW2025/blob/main/emotion_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ImDB をターゲットとした感情分析

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from collections import Counter
import re
import pickle
import os
from tqdm import tqdm

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [2]:
class IMDbDataset(Dataset):
    def __init__(self, texts, labels, vocab_to_idx, max_len=500):
        self.texts = texts
        self.labels = labels
        self.vocab_to_idx = vocab_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # テキストをインデックスに変換
        tokens = self.preprocess_text(text)
        indices = [self.vocab_to_idx.get(token, self.vocab_to_idx['<UNK>']) for token in tokens]

        # 長さ制限
        if len(indices) > self.max_len:
            indices = indices[:self.max_len]

        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

    def preprocess_text(self, text):
        # テキストの前処理
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        tokens = text.split()
        return tokens

In [3]:
class LSTMSentimentAnalyzer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes, dropout=0.5):
        super(LSTMSentimentAnalyzer, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers,
                           batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # bidirectional

    def forward(self, x, lengths=None):
        # x: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        # LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)

        # 最後の隠れ状態を使用（双方向なので結合）
        # hidden: (num_layers * 2, batch_size, hidden_dim)
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)  # (batch_size, hidden_dim * 2)

        # ドロップアウト
        hidden = self.dropout(hidden)

        # 分類層
        output = self.fc(hidden)  # (batch_size, num_classes)

        return output

In [4]:
# 諸々のヘルパー関数

def build_vocab(texts, min_freq=2, max_vocab_size=10000):
    """語彙を構築"""
    word_counts = Counter()

    for text in texts:
        tokens = preprocess_text(text)
        word_counts.update(tokens)

    # 頻度でソート
    vocab_items = word_counts.most_common(max_vocab_size - 4)  # 特殊トークン分を除く

    # 語彙辞書を作成
    vocab_to_idx = {
        '<PAD>': 0,
        '<UNK>': 1,
        '<SOS>': 2,
        '<EOS>': 3
    }

    for word, freq in vocab_items:
        if freq >= min_freq:
            vocab_to_idx[word] = len(vocab_to_idx)

    return vocab_to_idx

def preprocess_text(text):
    """テキストの前処理"""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = text.split()
    return tokens

def collate_fn(batch):
    """バッチ処理用の関数"""
    texts, labels = zip(*batch)

    # パディング
    texts = pad_sequence(texts, batch_first=True, padding_value=0)
    labels = torch.stack(labels)

    return texts, labels

def load_imdb_data():
    """IMDbデータセットを読み込み（実際の実装では適切なデータ読み込みを行う）"""
    # この部分は実際のデータセットに応じて実装
    # 例: torchtext.datasets.IMDB() を使用するか、
    # 手動でファイルから読み込む

    # サンプルデータ（実際の実装では置き換える）
    sample_texts = [
        "This movie was absolutely wonderful! Great acting and amazing plot.",
        "Terrible film. Waste of time and money. Very disappointed.",
        "Good movie with excellent cinematography and strong performances.",
        "Boring and predictable. Not worth watching.",
        "Outstanding film! Highly recommend to everyone.",
        "Poor acting and weak storyline. Not impressed."
    ]

    sample_labels = [1, 0, 1, 0, 1, 0]  # 1: positive, 0: negative

    # 実際の実装では以下のようにデータを読み込む
    # from torchtext.datasets import IMDB
    # train_iter, test_iter = IMDB(split=('train', 'test'))

    return sample_texts, sample_labels


In [5]:
def train_model(model, train_loader, criterion, optimizer, device):
    """1エポックの訓練"""
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []

    for batch_idx, (texts, labels) in enumerate(tqdm(train_loader, desc="Training")):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 予測値を取得
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(train_loader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')

    return avg_loss, accuracy, f1

def evaluate_model(model, test_loader, criterion, device):
    """モデルの評価"""
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for texts, labels in tqdm(test_loader, desc="Evaluating"):
            texts, labels = texts.to(device), labels.to(device)

            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')

    return avg_loss, accuracy, f1, predictions, true_labels

## メイン実行部分

In [8]:
from sklearn.model_selection import train_test_split

def main():
    # ハイパーパラメータ
    EMBEDDING_DIM = 128
    HIDDEN_DIM = 128
    NUM_LAYERS = 2
    NUM_CLASSES = 2
    DROPOUT = 0.5
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 10
    MAX_LEN = 500

    # データ読み込み
    print("データを読み込み中...")
    texts, labels = load_imdb_data()

    # 実際の実装では train/test 分割を行う
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

    # 動作チェック用
    #train_texts, train_labels = texts[:4], labels[:4]
    #test_texts, test_labels = texts[4:], labels[4:]

    # 語彙構築
    vocab_to_idx = build_vocab(train_texts)
    vocab_size = len(vocab_to_idx)
    print(f"語彙サイズ: {vocab_size}")

    # データセット作成
    train_dataset = IMDbDataset(train_texts, train_labels, vocab_to_idx, MAX_LEN)
    test_dataset = IMDbDataset(test_texts, test_labels, vocab_to_idx, MAX_LEN)

    # データローダー作成
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # モデル作成
    model = LSTMSentimentAnalyzer(vocab_size, EMBEDDING_DIM, HIDDEN_DIM,
                                 NUM_LAYERS, NUM_CLASSES, DROPOUT)
    model.to(device)

    # 損失関数とオプティマイザ
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # 訓練
    print("訓練開始...")
    best_f1 = 0

    for epoch in range(NUM_EPOCHS):
        # 訓練
        train_loss, train_acc, train_f1 = train_model(model, train_loader, criterion, optimizer, device)

        # 評価
        test_loss, test_acc, test_f1, predictions, true_labels = evaluate_model(model, test_loader, criterion, device)

        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}")
        print("-" * 60)

        # ベストモデルの保存
        if test_f1 > best_f1:
            best_f1 = test_f1
            torch.save(model.state_dict(), 'best_lstm_model.pth')
            print(f"ベストモデルを保存しました (F1: {best_f1:.4f})")

    # 最終評価
    print("\n最終評価:")
    print(classification_report(true_labels, predictions, target_names=['Negative', 'Positive']))

    # 語彙の保存
    with open('vocab.pkl', 'wb') as f:
        pickle.dump(vocab_to_idx, f)

if __name__ == "__main__":
    main()

データを読み込み中...
語彙サイズ: 7
訓練開始...


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 115.69it/s]


Epoch 1/10
Train Loss: 0.6723, Train Acc: 0.7500, Train F1: 0.7333
Test Loss: 0.6917, Test Acc: 0.5000, Test F1: 0.3333
------------------------------------------------------------
ベストモデルを保存しました (F1: 0.3333)


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 221.76it/s]


Epoch 2/10
Train Loss: 0.6522, Train Acc: 1.0000, Train F1: 1.0000
Test Loss: 0.6815, Test Acc: 1.0000, Test F1: 1.0000
------------------------------------------------------------
ベストモデルを保存しました (F1: 1.0000)


...

Epoch 8/10
Train Loss: 0.5942, Train Acc: 1.0000, Train F1: 1.0000
Test Loss: 0.6234, Test Acc: 1.0000, Test F1: 1.0000
------------------------------------------------------------


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 219.85it/s]


Epoch 9/10
Train Loss: 0.5554, Train Acc: 1.0000, Train F1: 1.0000
Test Loss: 0.6128, Test Acc: 1.0000, Test F1: 1.0000
------------------------------------------------------------


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 150.87it/s]


Epoch 10/10
Train Loss: 0.5830, Train Acc: 0.7500, Train F1: 0.7333
Test Loss: 0.6052, Test Acc: 1.0000, Test F1: 1.0000
------------------------------------------------------------

最終評価:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00         1
    Positive       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [9]:
def predict_sentiment(model, text, vocab_to_idx, device, max_len=500):
    """単一テキストの感情予測"""
    model.eval()

    # テキストの前処理
    tokens = preprocess_text(text)
    indices = [vocab_to_idx.get(token, vocab_to_idx['<UNK>']) for token in tokens]

    if len(indices) > max_len:
        indices = indices[:max_len]

    # テンソルに変換
    input_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

    sentiment = "Positive" if predicted.item() == 1 else "Negative"
    confidence = probabilities[0][predicted.item()].item()

    return sentiment, confidence

if __name__ == "__main__":
    # 保存されたモデルと語彙を読み込み
    with open('vocab.pkl', 'rb') as f:
        vocab_to_idx = pickle.load(f)

    model = LSTMSentimentAnalyzer(len(vocab_to_idx), 128, 128, 2, 2, 0.5)
    model.load_state_dict(torch.load('best_lstm_model.pth'))
    model.to(device)

    # テストテキスト
    test_text = "This movie was absolutely amazing! I loved every minute of it."
    sentiment, confidence = predict_sentiment(model, test_text, vocab_to_idx, device)
    print(f"Text: {test_text}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.4f})")

Text: This movie was absolutely amazing! I loved every minute of it.
Sentiment: Positive (Confidence: 0.5039)
